# _LangGraph_ Quickstart

In this quickstart you will create a multi-agent collaboration system with LangGraph and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the RAG triad of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/expositional/frameworks/langchain/langgraph_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need a [Tavily](https://app.tavily.com/home) and OpenAI key

In [ ]:
# !pip install trulens langgraph trulens-providers-openai openai matplotlib

In [ ]:
import os

os.environ["TAVILY_API_KEY"] = "tvly-dev-..."
os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["TRULENS_OTEL_TRACING"] = "1"

### Import from TruLens

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

### Define Tools

In [ ]:
from typing import Annotated

from langchain.load.dump import dumps
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langgraph.prebuilt import create_react_agent
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


tavily_tool = TavilySearchResults(max_results=5)

# Warning: This executes code locally, which can be unsafe when not sandboxed

repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = (
        f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    )
    return (
        result_str
        + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

### Define Nodes

In [ ]:
import json
from typing import Literal

from langchain_core.messages import BaseMessage
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END
from langgraph.graph import MessagesState
from langgraph.types import Command
from trulens.otel.semconv.trace import BASE_SCOPE
from trulens.apps.langgraph.inline_evaluations import inline_evaluation
import numpy as np
from trulens.core import Feedback
from trulens.core.feedback.selector import Selector
from trulens.providers.openai import OpenAI

from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="o4-mini-2025-04-16")

@instrument()
def orchestration_node(state: MessagesState) -> Command:
    # Find the most recent user query
    user_query = None
    user_index = None
    for i, msg in reversed(list(enumerate(state["messages"]))):
        if isinstance(msg, HumanMessage) and getattr(msg, "name", None) is None:
            user_query = msg.content
            user_index = i
            break
    if user_query is None:
        user_query = state["messages"][0].content
        user_index = 0

    # Find the most recent research answer after the user query
    research_answer = None
    for msg in reversed(state["messages"][user_index+1:]):
        if getattr(msg, "name", None) in ("web_researcher", "cortex_agents_researcher"):
            research_answer = msg.content
            break

    # Compose prompt for the LLM
    prompt = (
        "You are an orchestration agent for a multi-agent data assistant.\n"
        "Given the user query and the most recent research answer, decide if the research answer contains all the information needed to answer the query, or if more research is needed.\n"
        "- Respond 'done' if the research answer is sufficient to answer the query and no chart is requested or if a chart has been generated.\n"
        "- Respond 'search' if more information is needed.\n"
        "- Respond 'chart' if a chart is requested and the research answer is sufficient to generate a chart.\n"
        "\n"
        "User query:\n"
        f"{user_query}\n"
        "\n"
        "Most recent research answer:\n"
        f"{research_answer if research_answer else '[No research answer yet]'}\n"
        "\n"
        "Respond with only one word: 'cortex', 'web', or 'none'."
    )
    decision = llm.invoke(prompt).content.strip().lower()

    # Route accordingly
    if "search" in decision:
        return Command(update={}, goto="researcher")
    elif "done" in decision:
        return Command(update={}, goto=END)
    elif "chart" in decision:
        return Command(update={}, goto="chart_generator")
    else:
        return Command(update={}, goto="chart_generator")
    
def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto


def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )


llm = ChatOpenAI(model="gpt-4o")


def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto

## Research Node

In [ ]:
# Research agent and node
research_agent = create_react_agent(
    llm,
    tools=[tavily_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)

@instrument(
    span_type=SpanAttributes.SpanType.RETRIEVAL,
    attributes=lambda ret, exception, *args, **kwargs: {
        SpanAttributes.RETRIEVAL.QUERY_TEXT: args[0]["messages"][-1].content,
        SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: [
            json.loads(dumps(message)).get("kwargs", {}).get("content", "")
            for message in ret.update["messages"]
            if isinstance(message, ToolMessage)
        ]
        if hasattr(ret, "update")
        else "No tool call",
    },
)
def research_node(
    state: MessagesState,
) -> Command[Literal["orchestrator", END]]:
    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "orchestrator")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )
    return Command(
        update={
            # share internal message history of research agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )


## Chart Node

In [ ]:
# Chart generator agent and node
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
chart_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=make_system_prompt(
        "You can only generate charts. You are working with a researcher colleague."
    ),
)


@instrument(
    span_type="CHART_GENERATOR_NODE",
    attributes=lambda ret, exception, *args, **kwargs: {
        f"{BASE_SCOPE}.chart_node_input": args[0]["messages"][-1].content,
        f"{BASE_SCOPE}.chart_node_response": (
            ret.update["messages"][-1].content
            if ret and hasattr(ret, "update") and ret.update
            else "No update response"
        ),
    },
)
def chart_node(state: MessagesState) -> Command[Literal["researcher", END]]:
    result = chart_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "researcher")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    return Command(
        update={
            # share internal message history of chart agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )


chart_summary_agent = create_react_agent(
    llm,
    tools=[],  # Add image processing tools if available/needed.
    prompt=make_system_prompt(
        "You can only generate image captions. You are working with a researcher colleague and a chart generator colleague. "
        + "Your task is to generate a concise summary for the provided chart image saved at a local PATH, where the PATH should be and only be provided by your chart generator colleague. The summary should be no more than 3 sentences."
    ),
)


@instrument(
    span_type="CHART_SUMMARY_NODE",
    attributes=lambda ret, exception, *args, **kwargs: {
        f"{BASE_SCOPE}.summary_node_input": args[0]["messages"][-1].content,
        f"{BASE_SCOPE}.summary_node_output": ret.update["messages"][-1].content
        if hasattr(ret, "update")
        else "NO SUMMARY GENERATED",
    },
)
def chart_summary_node(
    state: MessagesState,
) -> Command[Literal["orchestrator", END]]:
    result = chart_summary_agent.invoke(state)
    # After captioning the image, we send control back (e.g., to the researcher)
    goto = get_next_node(result["messages"][-1], "orchestrator")
    # Wrap the output message in a HumanMessage to maintain consistency in the conversation flow.
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_summarizer"
    )
    return Command(
        update={"messages": result["messages"]},
        goto=goto,
    )

## Define the graph

In [ ]:
from IPython.display import Image
from IPython.display import display
from langgraph.graph import START
from langgraph.graph import StateGraph

workflow = StateGraph(MessagesState)
workflow.add_node("orchestrator", orchestration_node)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("chart_summarizer", chart_summary_node)

workflow.add_edge(START, "orchestrator")

graph = workflow.compile()

## Instrument graph for logging with TruLens

In [ ]:
class TruAgent:
    def __init__(self):
        self.graph = graph

    @instrument()
    def invoke_agent_graph(self, query: str) -> str:
            # rebuild the graph for each query
        # self.graph = workflow_compile()
            # Initialize state with proper message format
        state = {"messages": [HumanMessage(content=query)]}

            # Stream events with recursion limit
        events = self.graph.stream(
                state,
                {"recursion_limit": 100},
            )

            # Track all messages through the conversation
        all_messages = []
        for event in events:
                # Get the payload from the event
            _, payload = next(iter(event.items()))
            if not payload:  # Skip empty payloads
                continue

            messages = payload.get("messages")
            if not messages:
                continue
            all_messages.extend(messages)

            # Return the last message's content if available
        return (
            all_messages[-1].content
            if all_messages and hasattr(all_messages[-1], "content")
            else ""
            )


tru_agent = TruAgent()

## Initialize Feedback Function(s)

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.core.feedback.selector import Selector
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4.1")
traj_eval_provider = OpenAI(model_engine="o3")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on_context(collect_list=True)
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on_context(collect_list=False)
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

# Trajectory evaluations: step relevance of trace given user query
f_step_relevance = Feedback(
    provider.trajectory_step_relevance_with_cot_reasons, name="Step Relevance"
).on({
    "trace": Selector(trace_level=True),
})

# Trajectory evaluations: logical consistency of trace
f_logical_consistency = Feedback(
    provider.trajectory_logical_consistency_with_cot_reasons,
    name="Logical Consistency",
).on({
    "trace": Selector(trace_level=True),
})

# Trajectory evaluations: workflow efficiency of trace
f_workflow_efficiency = Feedback(
    provider.trajectory_workflow_efficiency_with_cot_reasons,
    name="Workflow Efficiency",
).on({
    "trace": Selector(trace_level=True),
    
})

### Register the Graph with TruLens

In [ ]:
from trulens.apps.app import TruApp

tru_recorder = TruApp(
   tru_agent,
    app_name="Multi-Agent Chart Generation",
    app_version="Base",
    feedbacks=[
     f_answer_relevance,
         f_context_relevance,
         f_groundedness,
         f_step_relevance,
         f_logical_consistency,
         f_workflow_efficiency,
    ],
)

### Run the Graph

In [ ]:
with tru_recorder as recording:
    # Run the multi-agent graph with a sample query
    result = tru_agent.invoke_agent_graph(
        "Generate a chart showing the yearly US GDP and National Debt over the last 100 years. Color the graph based on the political party of the president at the time."
    )

### Check results

In [ ]:
session.get_leaderboard()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session=session)

## Add in-line evals

In [ ]:
@instrument()
def orchestration_node(state: MessagesState) -> Command:
    # Find the most recent user query
    user_query = None
    user_index = None
    for i, msg in reversed(list(enumerate(state["messages"]))):
        if isinstance(msg, HumanMessage) and getattr(msg, "name", None) is None:
            user_query = msg.content
            user_index = i
            break
    if user_query is None:
        user_query = state["messages"][0].content
        user_index = 0

    # Find the most recent research answer after the user query
    research_answer = None
    for msg in reversed(state["messages"][user_index+1:]):
        if getattr(msg, "name", None) in ("web_researcher", "cortex_agents_researcher"):
            research_answer = msg.content
            break

    # Compose prompt for the LLM
    prompt = (
        "You are an orchestration agent for a multi-agent data assistant.\n"
        "Given the user query, the most recent research answer, and the evaluation score and explanation of the research if available.\n"
        "Decide if the research answer contains all the information needed to answer the query, or if more research is needed.\n"
        "- Respond 'done' if the research answer is sufficient to answer the query and no chart is requested or if a chart has been generated.\n"
        "- Respond 'search' if more information is needed.\n"
        "- Respond 'chart' if a chart is requested and the research answer is sufficient to generate a chart.\n"
        "\n"
        "User query:\n"
        f"{user_query}\n"
        "\n"
        "Most recent research answer:\n"
        f"{research_answer if research_answer else '[No research answer yet]'}\n"
        "\n"
        "Respond with only one word: 'cortex', 'web', or 'none'."
    )
    decision = llm.invoke(prompt).content.strip().lower()

    # Route accordingly
    if "search" in decision:
        return Command(update={}, goto="researcher")
    elif "done" in decision:
        return Command(update={}, goto=END)
    elif "chart" in decision:
        return Command(update={}, goto="chart_generator")
    else:
        return Command(update={}, goto="chart_generator")

In [ ]:
# eval for in-line evaluation
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance",
        criteria = "Context is fully relevant if it includes all of the information needed to answer the question, regardless of whether any visualizations or charts are included in the context."
    )
    .on({
            "question": Selector(
                span_type=SpanAttributes.SpanType.RETRIEVAL,
                span_attribute=SpanAttributes.RETRIEVAL.QUERY_TEXT,
            )
        }
    )
    .on({
            "context": Selector(
                span_type=SpanAttributes.SpanType.RETRIEVAL,
                span_attribute=SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS,
                collect_list=False
            )
        }
    )
    .aggregate(np.mean)
)

@inline_evaluation(f_context_relevance)
@instrument(
    span_type=SpanAttributes.SpanType.RETRIEVAL,
    attributes=lambda ret, exception, *args, **kwargs: {
        SpanAttributes.RETRIEVAL.QUERY_TEXT: args[0]["messages"][-1].content,
        SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: [
            json.loads(dumps(message)).get("kwargs", {}).get("content", "")
            for message in ret.update["messages"]
            if isinstance(message, ToolMessage)
        ]
        if hasattr(ret, "update")
        else "No tool call",
    },
)
def research_node(
    state: MessagesState,
) -> Command[Literal["orchestrator", END]]:
    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "orchestrator")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )
    return Command(
        update={
            # share internal message history of research agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )


## Recompile graph

In [ ]:
from IPython.display import Image
from IPython.display import display
from langgraph.graph import START
from langgraph.graph import StateGraph

workflow = StateGraph(MessagesState)
workflow.add_node("orchestrator", orchestration_node)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("chart_summarizer", chart_summary_node)

workflow.add_edge(START, "orchestrator")

graph_inline_evals = workflow.compile()

In [ ]:
class TruAgent:
    def __init__(self):
        self.graph = graph

    @instrument()
    def invoke_agent_graph(self, query: str) -> str:
            # rebuild the graph for each query
        # self.graph = workflow_compile()
            # Initialize state with proper message format
        state = {"messages": [HumanMessage(content=query)]}

            # Stream events with recursion limit
        events = self.graph.stream(
                state,
                {"recursion_limit": 100},
            )

            # Track all messages through the conversation
        all_messages = []
        for event in events:
                # Get the payload from the event
            _, payload = next(iter(event.items()))
            if not payload:  # Skip empty payloads
                continue

            messages = payload.get("messages")
            if not messages:
                continue
            all_messages.extend(messages)

            # Return the last message's content if available
        return (
            all_messages[-1].content
            if all_messages and hasattr(all_messages[-1], "content")
            else ""
            )


multi_agent_graph_inline_evals = TruAgent()

## Register new version of the agent

In [ ]:
from trulens.apps.app import TruApp

tru_recorder_inline_evals = TruApp(
    multi_agent_graph_inline_evals,
    app_name="Multi-Agent Chart Generation",
    app_version="in-line evals",
    feedbacks=[
     f_answer_relevance,
         f_context_relevance,
         f_groundedness,
         f_step_relevance,
         f_logical_consistency,
         f_workflow_efficiency,
    ],
)

## Test the new version

In [ ]:
with tru_recorder_inline_evals as recording:
    # Run the multi-agent graph with a sample query
    result = multi_agent_graph_inline_evals.invoke_agent_graph(
        "Generate a chart showing the yearly US GDP and National Debt over the last 100 years. Color the graph based on the political party of the president at the time."
    )

## Validate performance improvements

Wait some time for all evaluations to be completed.

In [ ]:
session.get_leaderboard()